In [1]:
import pandas as pd
import numpy as np

In [2]:
# define class
class ScoreType:
    def __init__(self, name, condition_function, scoring_function):
        self.name = name
        self.condition_function = condition_function
        self.scoring_function = scoring_function

    # method for checking whether the current hand 
    # meets the conditions for a certain score type
    def check_condition(self, hand):
        return self.condition_function(hand)
    
    # method for calculating score based on score 
    # type's unique logic
    def calculate_score(self, hand):
        return self.scoring_function(hand)
        
# establish each score type
chance = ScoreType(
    name = 'Chance',
    condition_function = lambda _: True,
    scoring_function = lambda hand: hand.sum()
)

ones = ScoreType(
    name = 'Ones',
    condition_function = lambda hand: 1 in hand,
    scoring_function = lambda hand: list(hand).count(1) * 1
)

twos = ScoreType(
    name = 'Twos',
    condition_function = lambda hand: 2 in hand,
    scoring_function = lambda hand: list(hand).count(2) * 2
)

threes = ScoreType(
    name = 'Threes',
    condition_function = lambda hand: 3 in hand,
    scoring_function = lambda hand: list(hand).count(3) * 3
)

fours = ScoreType(
    name = 'Fours',
    condition_function = lambda hand: 4 in hand,
    scoring_function = lambda hand: list(hand).count(4) * 4
)

fives = ScoreType(
    name = 'Fives',
    condition_function = lambda hand: 5 in hand,
    scoring_function = lambda hand: list(hand).count(5) * 5
)

sixes = ScoreType(
    name = 'Sixes',
    condition_function = lambda hand: 6 in hand,
    scoring_function = lambda hand: list(hand).count(6) * 6
)

three_kind = ScoreType(
    name = 'Three of a Kind',
    condition_function = lambda hand: np.unique(hand, return_counts = True)[1].max() >= 3,
    scoring_function = lambda hand: hand.sum()
)

four_kind = ScoreType(
    name = 'Four of a Kind',
    condition_function = lambda hand: np.unique(hand, return_counts = True)[1].max() >= 4,
    scoring_function = lambda hand: hand.sum()
)

full_house = ScoreType(
    name = 'Full House',
    condition_function = lambda hand: (
        (np.unique(hand, return_counts = True)[1].max() == 3) &
        (np.unique(hand, return_counts = True)[1].min() == 2)
    ),
    scoring_function = lambda _: 25
)

small_straight = ScoreType(
    name = 'Small Straight',
    condition_function = lambda hand: (
        all(number in hand for number in [1,2,3,4]) or
        all(number in hand for number in [2,3,4,5]) or
        all(number in hand for number in [3,4,5,6])
    ),
    scoring_function = lambda _: 30
)

large_straight = ScoreType(
    name = 'Large Straight',
    condition_function = lambda hand: (
        all(number in hand for number in [1,2,3,4,5]) or
        all(number in hand for number in [2,3,4,5,6])
    ),
    scoring_function = lambda _: 40
)

yahtzee = ScoreType(
    name = 'YAHTZEE',
    condition_function = lambda hand: list(hand).count(hand[0]) == 5,
    scoring_function = lambda _: 50
)

# collect all types in tuple
score_types = (
    chance,
    ones,
    twos,
    threes,
    fours,
    fives,
    sixes,
    three_kind,
    four_kind,
    full_house,
    small_straight,
    large_straight,
    yahtzee
)

In [3]:
# define class for score sheet
class ScoreSheet:
    def __init__(self, player_name = None, sheet = None):
        self.player_name = player_name
        self.sheet = dict(zip(
            [type.name for type in score_types],
            [None] * len(score_types)
        ))

    # update the sheet attribute with the score score / type
    def mark_score(self, chosen_score_type, chosen_score):
        self.sheet[chosen_score_type] = chosen_score

In [72]:
simulated_data = pd.DataFrame()

column_names = [
    'game',
    'round',
    'score_sheet_before_roll',
    'total_score_before_roll',
    'hand_before_roll',
    
    'roll_number',
    'dice_choices',
    'hand_after_roll',

    'remaining_score_types',
    'eligible_score_types',
    'chosen_score_type',
    'score',
    'total_score_after_roll',
    'score_sheet_after_roll'
]

In [149]:
def get_preround_data(game, round, score_sheet, total_score):
    return {
        'game':game,
        'round':round,
        'score_sheet_before_roll':score_sheet,
        'total_score_before_roll':total_score
    }

def get_roll_data(rolls_dict, roll, dice_choices, hand):
    rolls_dict['roll_number'].append(roll)
    rolls_dict['dice_choices'].append(dice_choices)
    rolls_dict['hand_after_roll'].append(hand)
    # return {
    #     'roll_number':roll_number,
    #     'dice_choices':[dice_choices],
    #     'hand_after_roll':[hand]
    # }

In [156]:
game = 0

# set up df for the game
game_data = {}

# instantiate a new score sheet and game variables
score_sheet = ScoreSheet()
total_score = 0 ### TARGET VARIABLE ###

# 13 rounds per game
for round in range(13):

    # define remaining score types, reset hand
    remaining_score_types = [x[0] for x in score_sheet.sheet.items() if x[1] == None]
    hand = np.array([], dtype = int)

    preround_dict = get_preround_data(game, round, score_sheet, total_score)
    rolls_dict = {
        'roll_number':[],
        'dice_choices':[],
        'hand_after_roll':[]
    }

    # loop for rolling dice
    for roll_number in range(3):

        # roll dice and add to hand
        roll = np.random.randint(1, 7, 5 - len(hand))
        hand = np.append(hand, roll)

        # on last turn, keep all dice in hand
        if roll_number == 2:
            continue
        
        # calculate probabilities
        ###################

        # choose dice to keep in hand
        keepers_idx = []
        dice_choices = [] ### TARGET VARIABLE ###
        for die_idx in range(5):    
            choice = np.random.choice(['drop','keep'])
            if choice == 'keep':
                keepers_idx.append(die_idx)
            dice_choices.append(choice)
        hand = hand[keepers_idx]

        # capture roll data
        get_roll_data(rolls_dict, roll, dice_choices, hand)

    # CHOOSING SCORE - reset variables
    eligible_score_types = []
    chosen_score_type = None
    score = 0

    # loop through all possible remaining scores
    for score_type in remaining_score_types:
        score_type_class = [x for x in score_types if x.name == score_type][0]
        
        # check if hand meets conditions for a given score
        if score_type_class.check_condition(hand):
            eligible_score_types.append(score_type)

    # if hand meets conditions for no score, choose a random and leave score 0
    if eligible_score_types == []:
        chosen_score_type = np.random.choice(remaining_score_types) ### TARGET VARIABLE ###
    # if hand does meet conditions for at least 1 score, choose a random and calculate points
    else:
        chosen_score_type = np.random.choice(eligible_score_types) ### TARGET VARIABLE ###
        score_type_class = [x for x in score_types if x.name == chosen_score_type][0]
        score = score_type_class.calculate_score(hand)

    # update total
    total_score += score

    # print results
    score_sheet.mark_score(chosen_score_type, score) 
    print(f'{chosen_score_type}: {score}')

    # finalize round df
    # results_df = pd.DataFrame({
    #     'remaining_score_types':[remaining_score_types],
    #     'eligible_score_types':[eligible_score_types],
    #     'chosen_score_type':chosen_score_type,
    #     'score':score,
    #     'total_score_after_roll':total_score,
    #     'score_sheet_after_roll':score_sheet
    # })
    # round_df = pd.concat([round_df, results_df])

    # capture results in game_df
    game_df = pd.concat([game_df, round_df])

# print total
print(f'-----\nTotal Score: {total_score}')

Fours: 4
Four of a Kind: 8
Ones: 2
Threes: 3
Chance: 14
Three of a Kind: 15
Twos: 2
Small Straight: 30
Sixes: 6
Fives: 5
Large Straight: 0
Full House: 0
YAHTZEE: 0
-----
Total Score: 89


In [157]:
rolls_dict

{'roll_number': [array([3, 5, 4, 2, 3]), array([5, 3])],
 'dice_choices': [['keep', 'drop', 'keep', 'keep', 'drop'],
  ['keep', 'keep', 'drop', 'keep', 'keep']],
 'hand_after_roll': [array([3, 4, 2]), array([3, 4, 5, 3])]}

In [151]:
get_roll_data(rolls_dict, roll, dice_choices, hand)

AttributeError: 'int' object has no attribute 'append'

In [155]:
test = {
    'roll_number':[],
    'dice_choices':[],
    'hand_after_roll':[]
}

# test = dict.fromkeys(['roll_number','dice_choices','hand_after_roll'], [])

test['roll_number'].append(1)
test['dice_choices'].append(dice_choices)
test['hand_after_roll'].append(hand)

test['roll_number'].append(2)
test['dice_choices'].append(dice_choices)
test['hand_after_roll'].append(hand)

test

{'roll_number': [1, 2],
 'dice_choices': [['drop', 'keep', 'keep', 'drop', 'keep'],
  ['drop', 'keep', 'keep', 'drop', 'keep']],
 'hand_after_roll': [array([2, 6, 1, 3, 2]), array([2, 6, 1, 3, 2])]}

In [154]:
{
    'roll_number':[],
    'dice_choices':[],
    'hand_after_roll':[]
}

{'roll_number': [], 'dice_choices': [], 'hand_after_roll': []}